In [2]:
# SELENIUM WEBDRIVER SCRIPT

import os
import time
import ctypes
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from itertools import dropwhile, islice

# set browser options
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

# start web browser
browser = webdriver.Chrome(options=options)

# get web code ; change URL to get the style you need - 'Outlined, Rounded, Sharp'
browser.get("https://fonts.google.com/icons?icon.set=Material+Symbols&icon.style=Sharp")
time.sleep(2)

# choose 'sort by name'
sort_by = browser.find_element(By.CSS_SELECTOR, '#main-content > icons-page > icons-sort-by > button')
actions = ActionChains(browser)
actions.click(sort_by)
actions.send_keys(Keys.ARROW_DOWN)
actions.send_keys(Keys.ENTER)
actions.perform()
time.sleep(2)

# scroll to the bottom of the page
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)

# fill dictionary with icon names and groups
print('... parsing in progress' + '\n')
names = {} # key = name, value = group

# <icons-group>
groups = browser.find_elements(By.TAG_NAME, 'icons-group')
for g in groups:
    header_ico = g.find_element(By.TAG_NAME, 'h2').text # group name    
    buttons_ico = g.find_elements(By.CLASS_NAME, 'ng-star-inserted')
    print('# ' + header_ico)
    for b in buttons_ico:      
        text = b.get_property("firstChild").get_attribute("innerText")
        names[text] = header_ico        

# save icon names to the file
print('\n' + '... saving the list' + '\n')
f = open('summary.txt', 'w')
for key, value in names.items():
    f.write(str(value) + ' - ' + str(key) + '\n')
f.close()

# show message box - OS Windows
message = ('Search is complete. ' + str(len(names)) + ' icons were found.' + 
           '\r\r' + 'The full list of icons from this web page has been' + 
             '\r' + 'saved to the //Script file folder/"summary.txt"' + 
           '\r\r' + 'If you need to change the settings of icons,' + 
             '\r' + 'it is better to do so after you click "Yes".' + 
           '\r\r' + 'Try to download any icon' + 
             '\r' + 'to initiate the automatiс process.' + 
           '\r\r' + 'Proceed?')

try:
    answer = ctypes.windll.user32.MessageBoxW(0, message, "Selenium", 4164)
    if answer == 6: pass
    elif answer == 7: browser.quit(); os._exit(0)
except WindowsError as win_err: print('An error occurred:\n{}'.format(win_err))

# <icons-side-nav-download-buttons>
wait = WebDriverWait(browser, 86400) # 24h timer
side_ndb = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'icons-side-nav-download-buttons')))
button_svg = side_ndb.find_element(By.TAG_NAME, 'a')

# add event listener, invoke once then automatically remove it
javascript = "let element = arguments[0]; \
              element.addEventListener('click', function() { \
                    let dummy = document.createElement('input'); \
                    dummy.setAttribute('type', 'hidden'); \
                    dummy.setAttribute('id', 'automation'); \
                    document.body.appendChild(dummy); \
                }, { once:true });"

# create the hidden element after click
browser.execute_script(javascript, button_svg)

# <input> id check up
try: # does the dummy attribute exist?
    wait.until(EC.presence_of_element_located((By.XPATH, '//input[@id="automation"]')))
except TimeoutException:
    print ("Time is up! Expected actions have not been performed.") 
    browser.quit(); os._exit(1)

# continue further if SVG button has been pushed...

# get selected icon
sel_icon = browser.find_element(By.CLASS_NAME, 'ng-star-inserted.selected')
sel_name = sel_icon.get_property("firstChild").get_attribute("innerText")

# get names that are below the selected one
icons = {k: names[k] for k in islice(dropwhile(lambda x: x!=sel_name, names), 1, None)}

# download loop
for key in icons.keys():
    #print(key)
    xpath = '//icons-group/div/button[@class="ng-star-inserted"]/span[1]/span[text()="{0}"]'.format(key)
    span_ico = browser.find_element(By.XPATH, xpath)
    browser.execute_script("arguments[0].click();", span_ico)
    time.sleep(1)
    button_svg.click()
    time.sleep(3)

# the end
print('... all done')

# close web browser
browser.close()

... parsing in progress

# Activities
# Android
# Audio & Video
# Brand
# Business & Payments
# Common actions
# Communication
# Hardware
# Home
# Household
# Maps
# Photo & Image
# Privacy & Security
# Social
# Text Formatting
# Transportation
# Travel
# UI actions

... saving to the file

... all done
